In [1]:
from detector.collision import check_collision
from detector.table import Table
import numpy as np
import cv2

In [75]:
table = Table('judd.webm')

In [147]:
table = Table('judd.webm')
i=0
coll_num = 0
while True:
    ret, frame = table.cap.read()
    if not ret:
        break
    frame = cv2.warpPerspective(frame, table.homography, table.size)
    out = frame.copy()
    mask = table._get_cloth_mask(frame)

    ctrs = table._get_ball_contours(mask)
    cv2.drawContours(out, ctrs, -1, (255,0,0), 2)
    ctr_colors = [-1]
    for idx, c in enumerate(ctrs):
        font = cv2.FONT_HERSHEY_SIMPLEX
        color = table.get_common_color(frame, c)
        color_name = table.predict_color(color)
        ctr_colors.append(color_name)
        
        if color_name == 'White':
            ctrs[0], ctrs[idx] = ctrs[idx], ctrs[0]
            ctr_colors.pop()
            ctr_colors[0] = color_name

        x, y, w, z = cv2.boundingRect(c)
        x -= 6
        y -= 6
        w += 10
        z += 10
        cv2.putText(out, f'{color_name}' , (x, y-5), font, 0.5, (int(color[0]), int(color[1]), int(color[2])), 2, cv2.LINE_AA)
        cv2.rectangle(out, (x, y), (x + w, y + z), (int(color[0]), int(color[1]), int(color[2])), 2)
    #print(ctr_colors)
    collision, col_idx = check_col(ctrs, ctr_colors)
    if collision:
        print(col_idx)
        print(ctr_colors[0], ctr_colors[col_idx[0]])
        coll_num += 1
        cont = ctrs[0]
        M = cv2.moments(cont)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        print('TRUE')
        cv2.putText(out, 'TEI!',(cx-100,cy-100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 5, cv2.LINE_AA)
    cv2.putText(out, '#Collisions: {}'.format(coll_num),(130,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3, cv2.LINE_AA)

     #frames.append(out)
    cv2.imshow("RuiBot", mask)
    i += 1
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

table.cap.release()
cv2.destroyAllWindows()

In [101]:
def check_col(contours):
    collision = False
    i=0
    col_dir = []
    col_pos = []
    if len(contours)>1:
        white = np.array(contours[0])
        for idx, ctr in enumerate(contours[1::]):
            colors = ctr.copy()
            #print(colors.shape, white.shape)
            # Squared difference between contours coordinates
            srdq = (white[:, None, :] - colors)**2
            # Euclidean distance between every white ball and colored balls contour points
            distances = np.sqrt(np.sum(srdq, axis=3))
            #print(distances.shape, white.shape, colors.shape)
            dist_min = 60
            if (distances < dist_min).any():
                #if (np.nonzero((distances < 20))[0].size) == 0:
                #    print(distances)
                col_pos.append(white[np.nonzero((distances < dist_min))[0][0]])
                M = cv2.moments(white)
                cx_white = int(M['m10']/M['m00'])
                cy_white = int(M['m01']/M['m00'])
                Mc = cv2.moments(colors)
                cx_color = int(Mc['m10']/Mc['m00'])
                cy_color = int(Mc['m01']/Mc['m00'])
                direction = np.array([cx_white - cx_color, cy_white - cy_color])
                #print(direction)
                direction = direction / np.linalg.norm(direction)
                #print(contour_colors[idx+1])
                col_dir.append(direction)
                collision = True
                #print(i)
        i+=1
    return (collision, col_dir, col_pos)

In [4]:
cv2.destroyAllWindows()

In [9]:
"""
COLLISION DETECTOR (WITH PLACEHOLDER TRACKER) TESTING WITH MASK

"""
from detector import Table
from tracker.placeholder_tracker import Tracker
from detector.collision import check_collision
table = Table('./judd.webm')
homography = table.homography
size = table.size
ccc = []

frames_vid = []
lists = []
ret, frame = table.cap.read()
frame = cv2.warpPerspective(frame, homography, size[:2])
mask = table._get_cloth_mask(frame)
#mask =  cv2.blur(mask,(2,2))
frame = cv2.bitwise_and(frame,frame, mask=mask)
ret2, frame2 = table.cap.read()
frame2 = cv2.warpPerspective(frame2, homography, size[:2])
mask = table._get_cloth_mask(frame2)
#mask =  cv2.blur(mask,(2,2))
frame2 = cv2.bitwise_and(frame2,frame2, mask=mask)
coll_num = 0
while table.cap.isOpened():
    if not ret2:
        break
    tracker = Tracker()
    tracker.track(frame, frame2, ycrcb = True)
    contours = tracker.contours
    ccc.append(contours)
    cv2.drawContours(frame, contours, -1, (255,0,0), 2)
    lists.append(frame)
    collision, col_idx, col_pos = check_col(contours)
    if collision:
        coll_num += 1
        cont = contours[0]
        M = cv2.moments(cont)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        print('TRUE')
        cv2.putText(frame, 'TEI!',(cx-100,cy-100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 5, cv2.LINE_AA)
        #cv2.putText(frame, 'TEI!',(100,160), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 5, cv2.LINE_AA)
    cv2.putText(frame, '#Collisions: {}'.format(coll_num),(130,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3, cv2.LINE_AA)
    cv2.namedWindow('frame')
    cv2.imshow('frame', frame)
    frame = frame2
    ret2, frame2 = table.cap.read()
    frame2 = cv2.warpPerspective(frame2, homography, size[:2])
    mask = table._get_cloth_mask(frame2)
    #mask =  cv2.blur(mask,(2,2))
    frame2 = cv2.bitwise_and(frame2,frame2, mask=mask)
    #frames_vid.append(frame2)
    #if ret2 is not None:
    #frame2 = cv2.warpPerspective(frame2, homography, size[:2])

    key = cv2.waitKey(1)
    if key == 27:
        break
cv2.destroyAllWindows()

TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE


error: OpenCV(4.5.3) /build/opencv/src/opencv-4.5.3/modules/imgproc/src/imgwarp.cpp:3144: error: (-215:Assertion failed) _src.total() > 0 in function 'warpPerspective'


In [127]:
"""
COLLISION DETECTOR (WITH PLACEHOLDER TRACKER) TESTING WITHOUT MASK

"""
from detector import Table
from tracker.placeholder_tracker import Tracker
from detector.collision import check_collision
table = Table('./judd.webm')
homography = table.homography
size = table.size
ccc = []

frames_vid = []
lists = []
ret, frame = table.cap.read()
frame = cv2.warpPerspective(frame, homography, size[:2])
#mask = table._get_cloth_mask(frame)
#mask =  cv2.blur(mask,(2,2))
#frame = cv2.bitwise_and(frame,frame, mask=mask)
ret2, frame2 = table.cap.read()
frame2 = cv2.warpPerspective(frame2, homography, size[:2])
#mask = table._get_cloth_mask(frame2)
#mask =  cv2.blur(mask,(2,2))
#frame2 = cv2.bitwise_and(frame2,frame2, mask=mask)
first_frame = frame.copy()
coll_num = 0
mask_collision = np.zeros_like(cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY))
while table.cap.isOpened():
    if not ret2:
        break
    tracker = Tracker()
    tracker.track(frame, frame2)
    contours = tracker.contours
    ccc.append(contours)
    cv2.drawContours(frame, contours, -1, (255,0,0), 2)
    lists.append(frame)
    collision, col_dir, col_pos = check_col(contours)
    if collision:
        print(col_dir)
        print(col_pos)
        coll_num += 1
        cont = contours[0]
        M = cv2.moments(cont)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        #print('TRUE')
        radius = 20
        center = tuple((col_pos[0].ravel() - col_dir[0]*2*radius).astype(np.int32))
        #cv2.circle(mask_collision, center, radius, 255, 5)
        cv2.ellipse(mask_collision, center, (int(radius*1.25), radius),90,0,360,255,-1)
        #cv2.circle(first_frame, tuple((col_pos[0].ravel() + col_dir[0]*2*radius).astype(np.int32)), radius, (255,0,0), 5)
        cv2.putText(frame, 'TEI!',(cx-100,cy-100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 5, cv2.LINE_AA)
        frames_vid.append(mask_collision)
        #cv2.putText(frame, 'TEI!',(100,160), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 5, cv2.LINE_AA)
    cv2.putText(frame, '#Collisions: {}'.format(coll_num),(130,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3, cv2.LINE_AA)
    cv2.namedWindow('frame')
    cv2.imshow('frame', frame)
    
    frame = frame2
    ret2, frame2 = table.cap.read()
    if ret2:
        frame2 = cv2.warpPerspective(frame2, homography, size[:2])
    #mask = table._get_cloth_mask(frame2)
    #mask =  cv2.blur(mask,(2,2))
    #frame2 = cv2.bitwise_and(frame2,frame2, mask=mask)
    #frames_vid.append(frame2)
    #if ret2 is not None:
    #frame2 = cv2.warpPerspective(frame2, homography, size[:2])

    key = cv2.waitKey(1)
    if key == 27:
        break
cv2.destroyAllWindows()

[array([0.74473519, 0.6673601 ])]
[array([[681, 843]], dtype=int32)]


In [106]:
cv2.destroyAllWindows()


In [192]:
def test_colision(table, path = './judd.webm'):
    #table = Table('./judd.webm')
    cap = cv2.VideoCapture(path)
    homography = table.homography
    size = table.size

    frames_vid = []
    lists = []
    ret, frame = cap.read()
    frame = cv2.warpPerspective(frame, homography, size[:2])

    ret2, frame2 = cap.read()
    frame2 = cv2.warpPerspective(frame2, homography, size[:2])
   
    first_frame = frame.copy()
    first_frame2 = frame.copy()
    coll_num = 0
    mask_collision = np.zeros_like(cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY))
    while table.cap.isOpened():
        if not ret2:
            break

        tracker = Tracker()
        tracker.track(frame, frame2)
        contours = tracker.contours
        
        cv2.drawContours(frame, contours, -1, (255,0,0), 2)
        lists.append(frame)
        collision, col_dir, col_pos = check_col(contours)
        color_name = ''
        if collision:
            print(col_dir)
            print(col_pos)
            coll_num += 1
            cont = contours[0]
            M = cv2.moments(cont)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            #print('TRUE')
            radius = 20
            center = tuple((col_pos[0].ravel() - col_dir[0]*2*radius).astype(np.int32))
            #cv2.circle(mask_collision, center, radius, 255, 5)
            cv2.ellipse(mask_collision, center, (int(radius*1.25), radius),90,0,360,255,-1)
            #cv2.circle(first_frame, tuple((col_pos[0].ravel() + col_dir[0]*2*radius).astype(np.int32)), radius, (255,0,0), 5)
            cv2.putText(frame, 'TEI!',(cx-100,cy-100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 5, cv2.LINE_AA)
            frames_vid.append(mask_collision)
            #cv2.putText(frame, 'TEI!',(100,160), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 5, cv2.LINE_AA)
            
            #first_frame2 = cv2.bitwise_and(first_frame2, first_frame2, mask=frames_vid[0])
            mask = table._get_cloth_mask(first_frame)
            mask = cv2.bitwise_and(mask, mask, mask=mask_collision)
            first_frame2 = mask_collision.copy()
            ctrs = table._get_ball_contours(mask)
            #first_frame2 = cv2.bitwise_and(first_frame2,first_frame2, mask=mask)
            for c in ctrs:
                font = cv2.FONT_HERSHEY_SIMPLEX
                color = table.get_common_color(first_frame, c)
                color_name = table.predict_color(color)
                print(color_name)
                x, y, w, z = cv2.boundingRect(c)
                x -= 6
                y -= 6
                w += 10
                z += 10
                cv2.putText(first_frame, f'{color_name}' , (x, y-5), font, 0.5, (int(color[0]), int(color[1]), int(color[2])), 2, cv2.LINE_AA)
                cv2.rectangle(first_frame, (x, y), (x + w, y + z), (int(color[0]), int(color[1]), int(color[2])), 2)
        cv2.putText(frame, '#Collisions: {}'.format(coll_num),(130,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3, cv2.LINE_AA)
        #cv2.namedWindow('frame')
        #cv2.imshow('frame', frame)
        
        frame = frame2
        ret2, frame2 = cap.read()
        if ret2:
            frame2 = cv2.warpPerspective(frame2, homography, size[:2])
        yield (collision, col_pos, col_dir, color_name, first_frame2)
        #mask = table._get_cloth_mask(frame2)
        #mask =  cv2.blur(mask,(2,2))
        #frame2 = cv2.bitwise_and(frame2,frame2, mask=mask)
        #frames_vid.append(frame2)
        #if ret2 is not None:
        #frame2 = cv2.warpPerspective(frame2, homography, size[:2])

        #key = cv2.waitKey(1)
        #if key == 27:
        #    break
    #cv2.destroyAllWindows()

In [193]:
table = Table('./judd.webm')
test = test_colision(table)
collision = False
while True:
    ret, frame = table.cap.read()
    if not ret:
        break
    frame = cv2.warpPerspective(frame, table.homography, table.size)
    out = frame.copy()
    mask = table._get_cloth_mask(frame)

    ctrs = table._get_ball_contours(mask)
    if not collision:
        collision, col_pos, col_dir, color_name, bu = next(test)
        if color_name: print(color_name)
    else:
        cv2.putText(out, f'Collided with {color_name}',(130,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3, cv2.LINE_AA)
    #frames.append(out)
    cv2.imshow("RuiBot", out)
    #i += 1
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cv2.destroyAllWindows()

[array([0.74473519, 0.6673601 ])]
[array([[681, 843]], dtype=int32)]
Red
Red


In [194]:
i3show(bu)

In [183]:
cv2.destroyAllWindows()

In [144]:
first_frame2 = first_frame.copy()
#first_frame2 = cv2.bitwise_and(first_frame2, first_frame2, mask=frames_vid[0])
mask = table._get_cloth_mask(first_frame2)
mask = cv2.bitwise_and(mask, mask, mask=frames_vid[0])
ctrs = table._get_ball_contours(mask)
#first_frame2 = cv2.bitwise_and(first_frame2,first_frame2, mask=mask)
for c in ctrs:
    font = cv2.FONT_HERSHEY_SIMPLEX
    color = table.get_common_color(first_frame2, c)
    color_name = table.predict_color(color)

    x, y, w, z = cv2.boundingRect(c)
    x -= 6
    y -= 6
    w += 10
    z += 10
    cv2.putText(first_frame2, f'{color_name}' , (x, y-5), font, 0.5, (int(color[0]), int(color[1]), int(color[2])), 2, cv2.LINE_AA)
    cv2.rectangle(first_frame2, (x, y), (x + w, y + z), (int(color[0]), int(color[1]), int(color[2])), 2)
#first_frame2 = cv2.bitwise_and(first_frame2,first_frame2, mask=mask)

i3show(first_frame2)

In [117]:
frames_vid[0].shape

(1000, 800, 3)

In [55]:
def i3show(img, win_name = 'Image'):
    cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
    cv2.imshow(win_name, img)
    while cv2.waitKey(0) != ord('q'):
        pass
    cv2.destroyAllWindows()

In [76]:
table = Table('./judd.webm')
homography = table.homography
size = table.size
ccc = []

frames_vid = []
lists = []
ret, frame = table.cap.read()
frame = cv2.warpPerspective(frame, homography, size[:2])
mask = table._get_cloth_mask(frame)
#mask =  cv2.blur(mask,(3,3))
frame = cv2.bitwise_and(frame,frame, mask=mask)
ret2, frame2 = table.cap.read()
frame2 = cv2.warpPerspective(frame2, homography, size[:2])
mask = table._get_cloth_mask(frame2)
#mask =  cv2.blur(mask,(3,3))
frame2 = cv2.bitwise_and(frame2,frame2, mask=mask)
i3show(frame2)

In [59]:
def saveVideo(frames, size=(640, 480), name='output', fps=1):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(name+'.mp4', fourcc, fps, (frames[0].shape[1], frames[0].shape[0]))
    for frame in frames:
        out.write(frame)
    out.release()

In [60]:
saveVideo(frames_vid, name = 'clothless', fps=30)

In [104]:
table.size

(800, 1000)

In [65]:
check_col(a)

True

In [6]:
a = np.array([4, 7])
b = np.array([1,3])
c = a-b
c = c/np.linalg.norm(c)
print(c)

[0.6 0.8]


In [219]:
def order_points(points):
    center = np.mean(points)
    shifted = points - center
    theta = np.arctan2(shifted[:, 0], shifted[:, 1])
    ind = np.argsort(theta)
    return points[ind]

fname = "./judd.webm"
cap = cv2.VideoCapture(fname)
cap_size = (int(cap.get(3)), int(cap.get(4)))
size = (800, 1000)
ret, frame = cap.read()

src = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
dst = cv2.Canny(src, 50, 100, None, 3)
blur = cv2.GaussianBlur(src,(5,5),0)
ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
cnts = cv2.findContours(th3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    #contours, hierarchy = cv2.findContours(th3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    #cv2.drawContours(image=image_copy, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
image_copy2 = frame.copy()
cnts = cnts[0]
cv2.drawContours(image=image_copy2, contours=cnts, contourIdx=-1, color=(0, 0, 255), thickness=4, lineType=cv2.LINE_AA) 
image_copy = frame.copy()
image_copy3 = frame.copy()
    # Find the contour with the maximum area.
c = max(cnts, key=cv2.contourArea)
cv2.drawContours(image=image_copy3, contours=c, contourIdx=-1, color=(0, 0, 255), thickness=4, lineType=cv2.LINE_AA) 
c = cv2.approxPolyDP(c, 0.02 * cv2.arcLength(c, True), True)
cv2.drawContours(image=image_copy, contours=c, contourIdx=-1, color=(0, 0, 255), thickness=20, lineType=cv2.LINE_AA)
pts_dst = np.array(
            [[0, 0], [size[0] - 1, 0], [size[0] - 1, size[1] - 1], [0, size[1] - 1]],
            dtype=float,
        )
corners = order_points(c[:, 0])
pts_dst = order_points(pts_dst)
h, _ = cv2.findHomography(corners, pts_dst)
final_img = cv2.warpPerspective(frame, h, size)
aux = cv2.copyMakeBorder(final_img, 40, 40, 10, 10, cv2.BORDER_CONSTANT,value=[255,255,255])
#lines = cv2.HoughLinesP(th3, 1, np.pi / 180, 150, None, 0, 0)
#for line in lines:
#	    x1, y1, x2, y2 = line[0]
#	    cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 3)
#cv2.namedWindow('Result Image', cv2.WINDOW_NORMAL)
vid_test = [frame, cv2.cvtColor(src, cv2.COLOR_GRAY2BGR), cv2.cvtColor(blur, cv2.COLOR_GRAY2BGR), cv2.cvtColor(th3, cv2.COLOR_GRAY2BGR), image_copy2, image_copy3, image_copy, aux]
cv2.namedWindow('Result Image', cv2.WINDOW_NORMAL)
cv2.imshow("Result Image", final_img)
while cv2.waitKey(0) != ord('q'):
        pass
cv2.destroyAllWindows()

In [200]:
print(final_img.shape)
frame.shape

(1000, 800, 3)


(1080, 1920, 3)

In [228]:
table = Table('./judd.webm')
listass = []
size = table.size
homography = table.homography
_, frame = table.cap.read()
frame = cv2.warpPerspective(frame, homography, size[:2])
mask = table._get_cloth_mask(frame)
ctrs = table._get_ball_contours(mask)

frame2 = cv2.bitwise_and(frame,frame, mask=mask)
cv2.drawContours(frame2, ctrs,-1, (255,0,0), 2)
listass.append(frame)
listass.append(frame2)

In [229]:
pad = np.full((1000, 20,3), 255).astype(np.uint8)
test_img2 = np.hstack([listass[0], pad ,listass[1]])
i3show(frame2)

In [215]:
cv2.destroyAllWindows()